# Модуль 3.1 — Custom Tools (кастомные инструменты)

**Цель:** научиться создавать безопасные инструменты с валидацией входов.

**Что сделаем:**
- установим библиотеки
- настроим ключ и base URL
- опишем инструмент с `args_schema`
- покажем безопасный вызов

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Безопасный кастомный инструмент

Мы создадим инструмент поиска товаров. Главное правило: **валидировать входы** и **не давать прямой доступ к системным вызовам**.

**Почему это важно:**
- агент может передать неожиданные значения
- без валидации легко получить ошибку или уязвимость

In [ ]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class SearchProductsInput(BaseModel):
    query: str = Field(description="Поисковый запрос на русском языке")
    max_results: int = Field(5, ge=1, le=20)

class ProductSearchTool(BaseTool):
    name = "product_search"
    description = "Поиск товаров в каталоге"
    args_schema = SearchProductsInput

    def _run(self, query: str, max_results: int = 5) -> str:
        # Безопасная имитация поиска (вместо прямого SQL)
        catalog = [
            "Беспроводные наушники AirSound",
            "Умная лампа BrightHome",
            "Фитнес‑браслет FitGo",
            "Электронная книга ReadLite",
        ]
        results = [item for item in catalog if query.lower() in item.lower()]
        return "\n".join(results[:max_results]) or "Ничего не найдено"

search_tool = ProductSearchTool()
print(search_tool.invoke({"query": "лампа", "max_results": 3}))

## Практические рекомендации

- Всегда используйте `args_schema` для валидации входов.
- Делайте ограничения по диапазонам (`ge`, `le`), чтобы защититься от больших запросов.
- Не давайте агенту прямого доступа к БД без параметризованных запросов.
- Логируйте вызовы инструмента для аудита и дебага.